In [1]:
import tensorflow as tf
gpu = tf.config.experimental.list_physical_devices('GPU')[0]
tf.config.experimental.set_memory_growth(gpu, True)
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2

In [2]:
loaded_model = tf.keras.models.load_model('mtm5.h5')

In [3]:
import pandas as pd

In [4]:
file_directory = 'document_data/'
classes = os.listdir(file_directory)
df = {'black':[],'good':[], 'grid':[], 'shaded':[], 'label': []}

for i in range(len(classes)):
    sub_dir = os.path.join(file_directory, classes[i])
    filename = os.listdir(sub_dir)
    for file_ in filename:
        file_path = os.path.join(sub_dir, file_)
        img = cv2.imread(file_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img , (360,360))
        _, img = cv2.threshold(img, 220, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        img = img / 255.0
        img = np.expand_dims(img, axis = -1)
        img = np.expand_dims(img, axis = 0)
        black_border_op, good_op, grid_op, shad_op  = loaded_model.predict(img)
        df['black'].append(np.round(black_border_op[0][0], 3))
        df['good'].append(np.round(good_op[0][0], 3))
        df['grid'].append(np.round(grid_op[0][0], 3))
        df['shaded'].append(np.round(shad_op[0][0], 3))
        df['label'].append(classes[i])
        
    print(f'completed {classes[i]}')

completed black_border
completed good
completed gridline
completed shaded


In [5]:
d = pd.DataFrame(df)
d

,black,good,grid,shaded,label
0,1.0,0.0,0.0,0.0,black_border
1,1.0,0.0,0.0,0.0,black_border
2,1.0,0.0,0.0,0.0,black_border
3,1.0,0.0,0.0,0.0,black_border
4,1.0,0.0,0.0,0.0,black_border
...,...,...,...,...,...
708,0.0,0.0,0.0,1.0,shaded
709,0.0,0.0,0.0,1.0,shaded
710,0.0,0.0,0.0,1.0,shaded
711,0.0,0.0,0.0,1.0,shaded


In [6]:
s  =pd.get_dummies(d.label,prefix="true",dtype= float )
s

,true_black_border,true_good,true_gridline,true_shaded
0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0
...,...,...,...,...
708,0.0,0.0,0.0,1.0
709,0.0,0.0,0.0,1.0
710,0.0,0.0,0.0,1.0
711,0.0,0.0,0.0,1.0


In [7]:
mat = pd.concat([d, s], axis=1)

In [8]:
mat.good.value_counts(), mat.true_good.value_counts()

(0.0    512
 1.0    201
 Name: good, dtype: int64,
 0.0    512
 1.0    201
 Name: true_good, dtype: int64)

In [9]:
mat.grid.value_counts(), mat.true_gridline.value_counts()

(0.0    605
 1.0    108
 Name: grid, dtype: int64,
 0.0    605
 1.0    108
 Name: true_gridline, dtype: int64)

In [10]:
mat.black.value_counts(), mat.true_black_border.value_counts()

(0.000    507
 1.000    201
 0.001      3
 0.999      1
 0.002      1
 Name: black, dtype: int64,
 0.0    511
 1.0    202
 Name: true_black_border, dtype: int64)

In [11]:
mat.shaded.value_counts(), mat.true_shaded.value_counts()

(0.000    511
 1.000    194
 0.999      4
 0.995      1
 0.892      1
 0.998      1
 0.978      1
 Name: shaded, dtype: int64,
 0.0    511
 1.0    202
 Name: true_shaded, dtype: int64)